In [11]:
import requests
import csv
from bs4 import BeautifulSoup
import time
import requests

In [2]:
pip install requests


Note: you may need to restart the kernel to use updated packages.


In [14]:
import requests
from bs4 import BeautifulSoup
import csv
import time

# Function to scrape comments from a post
def scrape_comments(post_soup):
    comments = post_soup.find_all('div', class_='comment')
    comment_list = []
    for comment in comments:
        author = comment.find('a', class_='author')
        author_name = author.text if author else "Unknown"
        comment_body = comment.find('div', class_='md').get_text(strip=True) if comment.find('div', class_='md') else "No content"
        comment_list.append({'author': author_name, 'comment': comment_body})
    return comment_list

# Function to scrape data from a single page
def scrape_page(url):
    headers = {'User-Agent': 'Mozilla/5.0'}
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.text, 'html.parser')
    posts = soup.find_all('div', attrs={'class': 'thing', 'data-domain': 'self.uktravel'})

    for post in posts:
        title = post.find('p', class_="title").text
        author = post.find('a', class_='author').text
        comments_link = post.find('a', class_='comments')['href']
        likes = post.find("div", attrs={"class": "score unvoted"}).text
        if likes == "•":
            likes = "None"

        # Fetching the post page
        post_page = requests.get(comments_link, headers=headers)
        post_soup = BeautifulSoup(post_page.text, 'html.parser')
        post_content = post_soup.find('div', class_='expando').get_text(strip=True) if post_soup.find('div', class_='expando') else "No content"
        
        # Fetching comments
        comments = scrape_comments(post_soup)

        # Save the post and its comments to CSV
        with open('output.csv', 'a', newline='', encoding='utf-8') as f:
            writer = csv.writer(f, delimiter='|', quoting=csv.QUOTE_MINIMAL)
            writer.writerow(['Post', title, author, likes, post_content])
            for comment in comments:
                writer.writerow(['Comment', comment['author'], '', '', comment['comment']])

    return soup, len(posts)

# Main scraping function
def main():
    base_url = "https://old.reddit.com/r/uktravel"
    counter = 0
    while counter < 100:
        soup, posts_count = scrape_page(base_url)
        if posts_count == 0:
            break
        next_button = soup.find("span", class_="next-button")
        if not next_button:
            break
        next_page_link = next_button.find("a").attrs['href']
        base_url = next_page_link
        counter += posts_count
        time.sleep(2)

if __name__ == "__main__":
    main()


PermissionError: [Errno 13] Permission denied: 'output.csv'